<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostProcessingRAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Installation
---



In [3]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Tue Jul 28 21:03:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Core(s) per socket:  2
Thread(s) per core:  2


In [5]:
%%time
%%capture
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

CPU times: user 673 ms, sys: 124 ms, total: 796 ms
Wall time: 7min 20s


In [ ]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

In [ ]:
from pathlib import Path
import os
from datetime import date

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import dask.dataframe as dd

In [ ]:
import cudf
import dask_cudf

#Post-processing process 

1.   Load and concatenate parquet files into dask_cudf
2.   Count 'Source"
1.   Merge with Shapefile (gdf)
1.   Create output data









In [ ]:
# Paths
pathShapefile = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/FFE/data/shapefile')
pathParquets = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

os.listdir(pathShapefile)

In [ ]:
def read_and_concatenate_parquets(path=pathParquets):
  L = []
  files = pathParquets.glob('*.parquet')
  number = 0
  for file in files:
    number += 1
    print(f"file loaded : {number}/{len(os.listdir(pathParquets))}")
    pqt = dd.read_parquet(file, engine='auto')
    # pqt = pd.read_parquet(file, columns=['source'])#, engine='auto')
    # pqtCudfDask = dask_cudf.from_dask_dataframe(pqt)
    L.append(pqt)
  df = dd.concat(L)
  return df

def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  count_df.to_parquet(str(pathShapefile) + '/' + f'CountBurn-{str(date.today())}).parquet', engine='auto', compression="GZIP")# could be datetime.now
  return count_df

def Merge(countDf,nameShapefile):
  # Shapefile
  gdfShape = gpd.read_file(pathShapefile / nameShapefile)
  gdfShape.insert(0, 'FID', range(0, len(gdf)))
  gdfShape.rename(columns={'FID': 'source'}, inplace=True)
  gdfShape = gdfShape[['source', 'geometry']]
  # gdf = pd.DataFrame(gdfShape)
  # cudf_gdf = cudf.DataFrame.from_pandas(gdf)
  # cudf_gdf.rename(columns={'FID': 'source'}, inplace=True)
  # cudf_gdf = cudf_gdf[['source', 'geometry']]
  # countDf as cudf
  # cudf_countDf = cudf.DataFrame.from_pandas(countDf)
  # merge
  merged = countDf.merge(gdfShape, on=['source'], how='left')
  return merged

def createShapefile(df):
  # df = df.to_pandas()
  gdf = gpd.GeoDataFrame(df, geometry='geometry')
  gdf.plot(column='count', cmap='hsv', legend=True)
  gdf.to_file(os.path.join(str(pathShapefile) + "/" + "Burn3000scenarioWellington.shp"))
  return gdf

In [ ]:
%%time
concatDf = read_and_concatenate_parquets()
countConcatDf = count_fid_occurences(concatDf)

In [70]:
countConcatDf

,source,count
0,35843,21497
1,24613,20607
2,28600,20301
3,22532,19735
4,68325,19578
...,...,...
69192,60394,1
69193,24376,1
69194,63508,1
69195,32941,1


In [71]:
gdfShape = gpd.read_file(pathShapefile / 'WellWHV_Buildings.shp')

In [72]:
gdfShape

,Replacemen,Combustibl,FloorArea,BLDG_ID,SA2_ID,NightOccup,IgnProbBld,geometry
0,271128.0,1,156.0,1,67,3.0,0.000040,"POLYGON ((1748253.040 5422598.875, 1748260.639..."
1,66044.0,1,38.0,2,75,1.0,0.000024,"POLYGON ((1752652.050 5423208.879, 1752658.230..."
2,56944.0,1,21.0,3,23,0.0,0.000101,"POLYGON ((1750820.799 5434554.465, 1750819.180..."
3,307626.0,1,177.0,4,38,3.0,0.000102,"POLYGON ((1750052.705 5432505.105, 1750056.199..."
4,342386.0,1,197.0,5,60,3.0,0.000081,"POLYGON ((1748074.536 5424848.405, 1748075.076..."
...,...,...,...,...,...,...,...,...
73730,330220.0,1,190.0,73731,33,3.0,0.000090,"POLYGON ((1752362.052 5434499.195, 1752375.208..."
73731,357378.0,1,132.0,73732,63,2.0,0.000060,"POLYGON ((1750067.567 5426183.099, 1750072.253..."
73732,939420.0,1,340.0,73733,23,5.0,0.000101,"POLYGON ((1750235.005 5434626.103, 1750243.300..."
73733,307626.0,1,177.0,73734,64,3.0,0.000288,"POLYGON ((1750454.255 5425527.410, 1750464.941..."


In [69]:
mergedDf = Merge(countConcatDf, 'WellWHV_Buildings.shp')

KeyError: ignored

In [ ]:
countShape = createShapefile(df)

In [ ]:
countShape.plot('count')

In [ ]:
eCu = cudf.read_parquet(p / 'Copy of edge_data.parquet')
ePd = pd.read_parquet(p / 'Copy of edge_data.parquet')

In [ ]:
type(eCu), type(ePd)

(cudf.core.dataframe.DataFrame, pandas.core.frame.DataFrame)

In [ ]:
eCu.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [ ]:
%time
test_eCu = eCu['IgnProb_bl'] * 100 / 4.252

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 16.9 µs


In [ ]:
%time
test_ePd = ePd['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [ ]:
eCuDask = dask_cudf.from_cudf(eCu, npartitions=2)

In [ ]:
%time
test_eCuDask = eCuDask['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_ePd = pd.concat([ePd,ePd])

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 51.3 µs


In [ ]:
%time
Concat_eCu = cudf.concat([eCu,eCu])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_eCuDask = dask_cudf.concat([eCuDask, eCuDask]).compute()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [ ]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Thu Jul  9 21:00:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    43W / 250W |   1789MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------